In [2]:
import requests
import os

class AirtableClient:
    def __init__(self, api_key, base_id, table_id):
        self.api_key = api_key
        self.base_id = base_id
        self.table_id = table_id
        self.base_url = f"https://api.airtable.com/v0/{self.base_id}/{self.table_id}"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

    def get_records(self, params=None):
        """Fetch records from the Airtable table."""
        response = requests.get(self.base_url, headers=self.headers, params=params)
        response.raise_for_status()
        return response.json()

    def update_record(self, record_id, fields):
        """
        Update a record in the Airtable table.

        :param record_id: The ID of the record to update.
        :param fields: A dictionary of fields to update.
        """
        url = f"{self.base_url}/{record_id}"
        data = {"fields": fields}
        response = requests.patch(url, headers=self.headers, json=data)
        response.raise_for_status()
        return response.json()


In [3]:
# Initialize
client = AirtableClient(api_key=os.getenv("AIRTABLE_API_KEY"), base_id="appPa8VJ4IHfm1V5O", table_id="tblF1QERP6FFNMnM1")

# Get records
records = client.get_records()
print(records)


{'records': [{'id': 'recDG3FwSxuvUL0jY', 'createdTime': '2025-05-16T17:13:10.000Z', 'fields': {'cv': '📄 CURRICULUM VITAE\n\nALEXANDER J. RUIZ\n📍 San Francisco, CA • 📞 (415) 987-6543 • ✉️ alex.ruiz.devops@protonmail.com • 🌐 linkedin.com/in/alexjruiz-devops • 💻 github.com/ajr-devops • ☁️ alexruiz.dev\n\n👨\u200d💼 Professional Summary\nSeasoned Principal DevOps Engineer with 12+ years of experience leading the design, implementation, and optimization of cloud-native infrastructure and DevOps pipelines. Proven track record in building robust, scalable, and secure platforms for enterprise applications across fintech, SaaS, and healthcare sectors. Specialized in Kubernetes, AWS/GCP/Azure, CI/CD architecture, observability, GitOps, and site reliability engineering (SRE). Passionate about infrastructure as code, cloud security, and mentoring cross-functional teams.\n\n🛠️ Core Competencies\nInfrastructure & Cloud Platforms:\n\nAWS (ECS, EKS, Lambda, CloudFormation, IAM, Route53, S3, RDS, CloudFr

In [19]:
# Update a record
updated = client.update_record("recDG3FwSxuvUL0jY", {"cv": "Updated cv"})
print(updated)

{'id': 'recDG3FwSxuvUL0jY', 'createdTime': '2025-05-16T17:13:10.000Z', 'fields': {'cv': 'Updated cv', 'id': '2'}}


In [22]:
class CVManager:
    def __init__(self, airtable_client):
        self.client = airtable_client

    def retrieve_all_cvs(self):
        """
        Retrieve all CV records and return a dict:
        { record_id: (cv, summarized_cv) }
        """
        records = self.client.get_records()
        result = {}
        for record in records.get("records", []):
            record_id = record["id"]
            fields = record.get("fields", {})
            cv = fields.get("cv", "")
            summarized_cv = fields.get("summarized_cv", "")
            result[record_id] = (cv, summarized_cv)
        return result

    def find_unsummarized_cv(self, target_cv):
        """
        Find a record where the cv matches and summarized_cv is empty.
        Returns (record_id, cv) or None if not found.
        """
        cvs = self.retrieve_all_cvs()
        for record_id, (cv, summarized) in cvs.items():
            if cv == target_cv and not summarized:
                return record_id, cv
        return None

    def update_summarized_cv(self, record_id, new_summary):
        """
        Update a specific record with a new summarized_cv.
        """
        return self.client.update_record(record_id, {"summarized_cv": new_summary})


In [24]:
cv_manager = CVManager(client)

# Find and update a CV
result = cv_manager.find_unsummarized_cv("Jane Doe Resume Content Here")
if result:
    record_id, cv_text = result
    summary = "Jane Doe is a project manager with 8 years of experience in logistics and retail tech."
    updated = cv_manager.update_summarized_cv(record_id, summary)
    print("Updated:", updated)
else:
    print("No matching unsummarized CV found.")

Updated: {'id': 'recDG3FwSxuvUL0jY', 'createdTime': '2025-05-16T17:13:10.000Z', 'fields': {'cv': 'Jane Doe Resume Content Here', 'summarized_cv': 'Jane Doe is a project manager with 8 years of experience in logistics and retail tech.'}}


In [26]:
class VacancyManager:
    def __init__(self, airtable_client):
        self.client = airtable_client

    def retrieve_all_vacancies(self):
        """
        Retrieve all vacancy records and return a dict:
        { record_id: (vacancy, summarized_vacancy) }
        """
        records = self.client.get_records()
        result = {}
        for record in records.get("records", []):
            record_id = record["id"]
            fields = record.get("fields", {})
            vacancy = fields.get("vacancy", "")
            summarized_vacancy = fields.get("summarized_vacancy", "")
            result[record_id] = (vacancy, summarized_vacancy)
        return result

    def find_unsummarized_vacancy(self, target_vacancy):
        """
        Find a record where the vacancy matches and summarized_vacancy is empty.
        Returns (record_id, vacancy) or None if not found.
        """
        vacancies = self.retrieve_all_vacancies()
        for record_id, (vacancy, summarized) in vacancies.items():
            if vacancy == target_vacancy and not summarized:
                return record_id, vacancy
        return None

    def update_summarized_vacancy(self, record_id, new_summary):
        """
        Update a specific record with a new summarized_vacancy.
        """
        return self.client.update_record(record_id, {"summarized_vacancy": new_summary})


In [31]:
vacancy_client = AirtableClient(api_key=os.getenv("AIRTABLE_API_KEY"), base_id="appPa8VJ4IHfm1V5O", table_id="tblfVZLqyJjb2SVHW")
vacancy_manager = VacancyManager(vacancy_client)

# Find and update a vacancy
result = vacancy_manager.find_unsummarized_vacancy("We are hiring a backend Python developer with 3+ years of experience...")
if result:
    record_id, vacancy_text = result
    summary = "Looking for a backend Python developer with 3+ years experience."
    updated = vacancy_manager.update_summarized_vacancy(record_id, summary)
    print("Updated:", updated)
else:
    print("No matching unsummarized vacancy found.")

Updated: {'id': 'recSQzHNjR0SVCja8', 'createdTime': '2025-05-16T17:13:10.000Z', 'fields': {'vacancy': 'We are hiring a backend Python developer with 3+ years of experience...', 'summarized_vacancy': 'Looking for a backend Python developer with 3+ years experience.'}}
